In [61]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pprint
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv1d, Conv2d,MaxPool1d, MaxPool2d, Module, Softmax, BatchNorm1d, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.nn.functional as F

# torchvision for pre-trained models
from torchvision import models

In [40]:
master_csv = pd.read_csv(r"C:\Users\sonam\University of Canberra\4th Semester\Capstone\Models\Sub CSV.csv")
master_csv.head()

,filename,session,subject,emotion
0,cz_eeg_data_1.csv,1,cz,1
1,cz_eeg_data_2.csv,1,cz,2
2,cz_eeg_data_3.csv,1,cz,3
3,cz_eeg_data_4.csv,1,cz,0
4,cz_eeg_data_5.csv,1,cz,2


In [41]:
cwd = os.getcwd() 
folder_path = os.path.join(cwd,"cz_200")

eeg_data=[]
for filename in tqdm(master_csv['filename']):
    file_path = os.path.join(folder_path, filename)
    file = pd.read_csv(file_path)
    
    # Drop the first row (column headers)
    file = file.iloc[1:]
    
    # Drop rows below row 9600
    file = file.iloc[:6002]
    
    file = file.astype('float32')
    eeg_values = file.drop(columns=['time']).values  # Assuming 'time' column is not needed
    
#      # Min-max normalization
#     min_val = np.min(eeg_values)
#     max_val = np.max(eeg_values)
#     eeg_values_normalized = (eeg_values - min_val) / (max_val - min_val)
    
        
#     # Calculate mean and standard deviation for Z-score normalization
#     mean = np.mean(eeg_values, axis=0)
#     std_dev = np.std(eeg_values, axis=0)
    
#     # Apply Z-score normalization
#     eeg_values_normalized = (eeg_values - mean) / std_dev
    
    
    # Transpose the EEG data
    eeg_values = eeg_values.T
    
    eeg_data.append(eeg_values)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:12<00:00,  1.89it/s]


In [42]:
# converting the list to numpy array
eeg_numpy = np.array(eeg_data)
eeg_numpy.shape

#%%

(24, 62, 6002)

In [43]:
eeg_numpy.shape

(24, 62, 6002)

In [65]:
# defining the target
label_emotion = master_csv['emotion'].values
label_emotion

array([1, 2, 3, 0, 2, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 3, 2, 2, 3, 3, 0, 3,
       0, 3], dtype=int64)

In [66]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(eeg_numpy, label_emotion, test_size = 0.25, random_state = 13, stratify=label_emotion)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((18, 62, 6002), (18,)), ((6, 62, 6002), (6,)))

In [67]:
# converting training images into torch format

train_x = torch.from_numpy(train_x)


In [68]:
# converting the target into torch format
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)


In [69]:
# # shape of training data
train_x.shape, train_y.shape

(torch.Size([18, 62, 6002]), torch.Size([18]))

In [70]:

val_x = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)


In [81]:
# shape of validation data
val_x.shape, val_y.shape

(torch.Size([6, 62, 6002]), torch.Size([6]))

In [84]:
class CNN(torch.nn.Module):
    def __init__(self, in_channels=1, num_classes=3):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.ZeroPad2d((1, 2, 1, 2)),
            nn.Conv2d(in_channels, 64, kernel_size=4, stride=1),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.ZeroPad2d((1, 2, 1, 2)),
            nn.Conv2d(64, 128, kernel_size=4, stride=1),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.ZeroPad2d((1, 2, 1, 2)),
            nn.Conv2d(128, 256, kernel_size=4, stride=1),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.ZeroPad2d((1, 2, 1, 2)),
            nn.Conv2d(256, 64, kernel_size=4, stride=1),
            nn.ReLU()
        )
        
        
        # Calculate the number of features after convolutional layers
        self.num_features = self._calculate_num_features((62, 6002))


        
        # Adjusted the input size of the first linear layer
        self.lin1 = nn.Linear(self.num_features, 1024)  # Adjusted the input size
        self.lin2 = nn.Linear(1024, num_classes)

        
    def _calculate_num_features(self, input_dim):
        input_tensor = torch.zeros(1, *input_dim)
        conv_output = self.conv1(input_tensor)
        conv_output = self.conv2(conv_output)
        conv_output = self.conv3(conv_output)
        conv_output = self.conv4(conv_output)
        num_features = conv_output.view(1, -1).size(1)
        return num_features
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)

        x = x.flatten(start_dim=1)
        x = self.lin1(x)
        x = self.lin2(x)
        return x

In [85]:
# defining the model
model = CNN()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
#if torch.cuda.is_available():
device='cpu'
model = model.to(device)
criterion = criterion.to(device)

print(model)

# batch size of the model
batch_size = 200

# number of epochs to train the model
n_epochs = 10


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 97550073856 bytes.

In [ ]:
for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
        
    permutation = torch.randperm(train_x.size()[0])

    training_loss = []
    for i in tqdm(range(0,train_x.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]
        
        # Convert batch_y to Long data type
        batch_y = batch_y.long()  # Convert to Long data type
        
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        # in case you wanted a semi-full example
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

In [ ]:
# prediction for training set
prediction = []
target = []
permutation = torch.randperm(train_x.size()[0])
for i in tqdm(range(0,train_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = train_x[indices], train_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i],prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))


In [115]:
# prediction for validation set
prediction_val = []
target_val = []
permutation = torch.randperm(val_x.size()[0])
for i in tqdm(range(0,val_x.size()[0], batch_size)):
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = val_x[indices], val_y[indices]

    #if torch.cuda.is_available():
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    with torch.no_grad():
        output = model(batch_x)

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions = np.argmax(prob, axis=1)
    prediction_val.append(predictions)
    target_val.append(batch_y)
    
# validation accuracy
accuracy_val = []
for i in range(len(prediction_val)):
    accuracy_val.append(accuracy_score(target_val[i],prediction_val[i]))
    
print('validation accuracy: \t', np.average(accuracy_val))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.92it/s]

validation accuracy: 	 0.3333333333333333
